In [161]:
import geopandas as gpd
import pandas as pd
import folium
import pydeck as pdk
import warnings
warnings.filterwarnings('ignore')


def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [4]:
indo = gpd.read_file('FIX_INA_Pupuk.shp')


,NAME_1,AREA,MIN,MAX,RANGE,MEAN,STD,SUM,Provinsi,Dosis_Pupu,Data_untuk,Total_kebu,geometry
0,Bangka Belitung,1.319304,0.45,2.00,1.55,1.517072,0.160959,2.480238e+06,Bangka Belitung,3.563.010,"10.912,605",74.525,"MULTIPOLYGON (((107.96173 -3.41100, 107.96220 ..."
1,Gorontalo,0.968277,0.45,2.00,1.55,1.620381,0.144516,1.944283e+06,Gorontalo,3.935.250,"11.890,725",28.325,"MULTIPOLYGON (((122.81573 0.84676, 122.81579 0..."
2,Riau,7.212746,0.45,2.00,1.55,1.601386,0.137029,1.431328e+07,Riau,8.743.866,"28.961,973",910.125,"MULTIPOLYGON (((101.64432 -0.82552, 101.64344 ..."
3,Jakarta Raya,0.052157,0.45,2.25,1.80,1.205084,0.095654,7.788820e+04,Jakarta Raya,1.376.100,"4.128,300",0.000,"MULTIPOLYGON (((106.73563 -6.04014, 106.73548 ..."
4,Kepulauan Riau,0.534465,0.45,1.85,1.40,1.515761,0.262557,1.003905e+06,Kepulauan Riau,2.852.190,"8.604,495",15.975,"MULTIPOLYGON (((105.25355 -1.29511, 105.25032 ..."


In [7]:
len(indo)

34

In [185]:
GREAT_CIRCLE_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/flights.json"  # noqa

df_n = pd.read_json(GREAT_CIRCLE_LAYER_DATA)

# Use pandas to prepare data for tooltip
df_n["from_name"] = df_n["from"].apply(lambda f: f["name"])
df_n["to_name"] = df_n["to"].apply(lambda t: t["name"])

# # Define a layer to display on a map
# layer = pdk.Layer(
#     "GreatCircleLayer",
#     df,
#     pickable=True,
#     get_stroke_width=12,
#     get_source_position="from.coordinates",
#     get_target_position="to.coordinates",
#     get_source_color=[64, 255, 0],
#     get_target_color=[0, 128, 200],
#     auto_highlight=True,
# )

# # Set the viewport location
# view_state = pdk.ViewState(latitude=50, longitude=-40, zoom=1, bearing=0, pitch=0)

# # Render
# r = pdk.Deck(
#     layers=[layer],
#     initial_view_state=view_state,
#     tooltip={"text": "{from_name} to {to_name}"},
# )
# r.picking_radius = 10

# r.to_html("great_circle_layer.html")

df_n.iloc[0]['from']['coordinates']

[-122.38347034444931, 37.61702508680534]

In [11]:
m = folium.Map(location=[50, -40], zoom_start=1)

for _, row in df.iterrows():
    from_coords = row["from"]["coordinates"]
    to_coords = row["to"]["coordinates"]
    from_name = row["from_name"]
    to_name = row["to_name"]

    folium.PolyLine(locations=[from_coords, to_coords],
                    color='blue',
                    weight=12,
                    tooltip=f'{from_name} to {to_name}').add_to(m)


m.save("great_circle_map_folium.html")

In [24]:
myl = list(indo.MEAN)
myl.sort()
print(myl)
med = len(myl) // 2
print("Median of Mean Data: {}".format(myl[med]))

[1.20508413669, 1.38054625314, 1.44404702931, 1.50355371583, 1.50389812964, 1.50595387766, 1.51236026076, 1.51467413353, 1.51576116908, 1.51706247814, 1.51707161853, 1.53202599956, 1.53222020453, 1.53515771432, 1.56428020236, 1.56512209308, 1.5754733483, 1.57652300024, 1.59237447767, 1.59986514131, 1.60138642408, 1.62038136136, 1.63154578265, 1.6432110722, 1.66180367047, 1.68508420633, 1.83200111759, 1.89718463245, 1.92323040858, 1.98236163249, 1.99016524099, 2.00364606463, 2.04198140907, 2.0522960072]
Median of Mean Data: 1.57652300024


In [39]:
pupuk, demand = list(indo.Dosis_Pupu), list(indo.Total_kebu)

pupuk_conv = [int(x.replace('.', '')) for x in pupuk]
indo.Dosis_Pupu = pupuk_conv
pupuk_conv.sort()
demand.sort()

print(pupuk_conv)
print(demand)
print("Median of Pupuk Hortikultura Data: {}".format(pupuk_conv[med]))
print("Median of Pupuk Perkebunan Data: {}".format(demand[med]))

[1376100, 2852190, 3563010, 3609540, 3935250, 5710320, 5931090, 6077610, 6609240, 7280460, 8437770, 8587026, 8743866, 9080280, 9529740, 9908910, 9931689, 10534590, 11451330, 11498850, 11748330, 11823618, 13888710, 21968415, 27683041, 29068380, 32414580, 35706330, 56579490, 61079286, 70496512, 142374323, 188520750, 221509530]
[0.0, 12.55, 15.975, 23.7, 28.325, 28.6, 29.95, 31.775, 38.25, 55.625, 57.9, 65.275, 71.9, 74.525, 75.65, 86.925, 91.7, 98.675, 99.3, 107.575, 133.375, 163.45, 174.075, 182.6, 186.875, 207.475, 228.275, 367.725, 411.6, 488.225, 539.75, 577.9, 659.925, 910.125]
Median of Pupuk Hortikultura Data: 10534590
Median of Pupuk Perkebunan Data: 98.675


In [70]:
a = set(indo.loc[(indo.MEAN < myl[med]) & 
     ((indo.Dosis_Pupu > pupuk_conv[med]) & 
     (indo.Total_kebu > demand[med]))].Provinsi)
print(list(a))
# tujuan prior

b = set(indo.loc[(indo.MEAN < myl[med]) & 
     ((indo.Dosis_Pupu > pupuk_conv[med]) |
     (indo.Total_kebu > demand[med]))].Provinsi)
# tujuan prior_2
c = list(b.difference(a))

print(c)

['Sumatera Barat', 'Bengkulu', 'Aceh', 'Sumatera Utara', 'Kalimantan Barat']
['Kalimantan Tengah', 'Sulawesi Utara', 'Kalimantan Timur', 'Kalimantan Selatan', 'Sumatera Selatan', 'Sulawesi Barat']


In [72]:
d = list(indo.loc[(indo.MEAN > myl[med]) & 
     ((indo.Dosis_Pupu > pupuk_conv[med]) | 
     (indo.Total_kebu > demand[med]))].Provinsi)
# distributhor
print(d)


['Riau', 'Sulawesi Selatan', 'Lampung', 'Jawa Tengah', 'Nusa Tenggara Barat', 'Jambi', 'Jawa Timur', 'Yogyakarta', 'Nusa Tenggara Timur', 'Jawa Barat', 'Sulawesi Tenggara']


In [86]:
lon1 = indo.iloc[0].geometry.centroid.x
lat1 = indo.iloc[0].geometry.centroid.y
lon2 = indo.iloc[1].geometry.centroid.x
lat2 = indo.iloc[1].geometry.centroid.y
print(lon1, lat1, lon2, lat2)

106.55274196400221 -2.44845864447203 122.37615124729896 0.6868471160930348


In [150]:
df["from_name"] = df["from"].apply(lambda f: f["name"])
df["to_name"] = df["to"].apply(lambda t: t["name"])

# Define a layer to display on a map
layer = pdk.Layer(
    "GreatCircleLayer",
    df,
    pickable=True,
    get_stroke_width=12,
    get_source_position=[lon1, lat1],
    get_target_position=[lon2 , lat2],
    get_source_color=[255, 0, 0],
    get_target_color=[255, 255, 255],
    auto_highlight=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=-0.7893, longitude=113.9213, zoom=2, bearing=0, pitch=0)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "ini ke itu"},
)
r.picking_radius = 5

r.to_html("great_circle_layer.html")



TypeError: 'NoneType' object is not subscriptable

In [ ]:
prior1 = ['Sumatera Barat', 'Bengkulu', 'Aceh', 'Sumatera Utara', 'Kalimantan Barat']
prior2 = ['Kalimantan Tengah', 'Sulawesi Utara', 'Kalimantan Timur', 'Kalimantan Selatan', 'Sumatera Selatan', 'Sulawesi Barat']
distributor = ['Riau', 'Sulawesi Selatan', 'Lampung', 'Jawa Tengah', 'Nusa Tenggara Barat', 'Jambi', 'Jawa Timur', 'Yogyakarta', 'Nusa Tenggara Timur', 'Jawa Barat', 'Sulawesi Tenggara']

# layer[i]1 -> distributor[i] to prior1
# layer[i]2 -> distributor[i] to prior2

# df structures 
# from, from_coords, to, to_coords

list_of_layers_1 = []
list_of_layers_2 = []
df = None

for i in range(1):
    from_x = float(indo.loc[indo.Provinsi == distributor[i]].geometry.centroid.x)
    from_y = float(indo.loc[indo.Provinsi == distributor[i]].geometry.centroid.y)
                    
    to_coords_1 = []
    for j in range(len(prior1)):
        to_coords_1.append([float(indo.loc[indo.Provinsi == prior1[j]].geometry.centroid.x), 
                          float(indo.loc[indo.Provinsi == prior1[j]].geometry.centroid.y)])
    print(distributor[i], len(from_coords), len(prior1), len(to_coords_1), to_coords_1)
    df = pd.DataFrame({
        'from' : distributor[i],
        'from_x' : from_x,
        'from_y' : from_y,
        'to' : prior1,
        'to_coords' : to_coords_1
    })

layer = pdk.Layer(
    "GreatCircleLayer",
    df,
    pickable=True,
    get_stroke_width=12,
    get_source_position= [df.from_x, df.from_y],
    get_target_position = df.to_coords,
    get_source_color=[255, 0, 0],
    get_target_color=[255, 255, 255],
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=-0.7893, longitude=113.9213, zoom=2, bearing=0, pitch=0)

r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "ini ke itu"},
)
r.picking_radius = 5

r.to_html("asdasdad.html")

# df1.head()

# df = pd.DataFrame({
#     'a' : [1,2,3,4]
# })

# df2 = pd.DataFrame({
#     'a' : [1,2,3,4]
# })

# df.iloc[0]['from']

In [202]:
layer = pdk.Layer(
    "GreatCircleLayer",
    df,
    pickable=False,
    get_stroke_width=15,
    get_source_position= "[from_x, from_y]",
    get_target_position = "to_coords",
    get_source_color=[64, 255, 0],
    get_target_color=[0, 128, 200],
)

view_state = pdk.ViewState(latitude=df.iloc[0].from_y, longitude=df.iloc[0].from_x, zoom=5, bearing=0, pitch=0)

r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
)
r.picking_radius = 5

r.to_html("Riau.html")